In [74]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [79]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} conda-forge::fiona

^C


In [7]:
regions.head()

,"id,code,name,slug"
0,"1,01,Guadeloupe,guadeloupe"
1,"2,02,Martinique,martinique"
2,"3,03,Guyane,guyane"
3,"4,04,""La Réunion"",""la reunion"""
4,"5,06,Mayotte,mayotte"


In [10]:
referendum = pd.read_csv("data/referendum.csv", sep=";")
regions = pd.read_csv("data/regions.csv", sep=",")
departments = pd.read_csv("data/departments.csv", sep=",")

df_merged = pd.merge(regions, departments, right_on='region_code',left_on='code', suffixes=('_reg', '_dep'))
regions_and_departments = df_merged[['code_reg', 'name_reg', 'code_dep', 'name_dep']]


In [31]:
regions_and_departments.shape

(109, 4)

In [47]:
strg='COM'
#pd.merge(referendum, regions_and_departments, left_on='Department code', right_on='code_dep').query("code_reg.str.contains(@strg)")
df=pd.merge(referendum, regions_and_departments, left_on='Department code', right_on='code_dep',how='right')

In [62]:
regions_and_departments

,code_reg,name_reg,code_dep,name_dep
0,01,Guadeloupe,971,Guadeloupe
1,02,Martinique,972,Martinique
2,03,Guyane,973,Guyane
3,04,La Réunion,974,La Réunion
4,06,Mayotte,976,Mayotte
...,...,...,...,...
104,COM,Collectivités d'Outre-Mer,984,Terres australes et antarctiques françaises
105,COM,Collectivités d'Outre-Mer,986,Wallis et Futuna
106,COM,Collectivités d'Outre-Mer,987,Polynésie française
107,COM,Collectivités d'Outre-Mer,988,Nouvelle-Calédonie


In [70]:
referendum['Department code'] = referendum['Department code'].str.zfill(2)
referendum_and_areas=pd.merge(referendum, regions_and_departments, left_on='Department code', right_on='code_dep')

In [71]:
referendum_and_areas

,Department code,Department name,Town code,Town name,Registered,Abstentions,Null,Choice A,Choice B,code_reg,name_reg,code_dep,name_dep
0,01,AIN,1,L'Abergement-Clémenciat,592,84,9,154,345,84,Auvergne-Rhône-Alpes,01,Ain
1,01,AIN,2,L'Abergement-de-Varey,215,36,5,66,108,84,Auvergne-Rhône-Alpes,01,Ain
2,01,AIN,4,Ambérieu-en-Bugey,8205,1698,126,2717,3664,84,Auvergne-Rhône-Alpes,01,Ain
3,01,AIN,5,Ambérieux-en-Dombes,1152,170,18,280,684,84,Auvergne-Rhône-Alpes,01,Ain
4,01,AIN,6,Ambléon,105,17,1,35,52,84,Auvergne-Rhône-Alpes,01,Ain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36560,95,VAL D'OISE,676,Villers-en-Arthies,382,64,5,87,226,11,Île-de-France,95,Val-d'Oise
36561,95,VAL D'OISE,678,Villiers-Adam,595,71,4,191,329,11,Île-de-France,95,Val-d'Oise
36562,95,VAL D'OISE,680,Villiers-le-Bel,13269,4083,224,5546,3416,11,Île-de-France,95,Val-d'Oise
36563,95,VAL D'OISE,682,Villiers-le-Sec,95,11,0,40,44,11,Île-de-France,95,Val-d'Oise


In [73]:
df_grouped = referendum_and_areas.groupby('code_reg').agg({
    'name_reg': 'first',  # Keep the fisrt name 
    'Registered': 'sum',
    'Abstentions': 'sum',
    'Null': 'sum',
    'Choice A': 'sum',
    'Choice B': 'sum'
})
df_grouped

,name_reg,Registered,Abstentions,Null,Choice A,Choice B
code_reg,,,,,,
11,Île-de-France,6942361,1515884,85997,2536620,2803860
24,Centre-Val de Loire,1801366,326560,28827,590890,855089
27,Bourgogne-Franche-Comté,1997925,358321,33671,668246,937687
28,Normandie,2372779,426075,35513,816972,1094219
32,Hauts-de-France,4210444,866555,60601,1411257,1872031
44,Grand Est,3857089,783348,56951,1088684,1928106
52,Pays de la Loire,2605438,412509,47677,922468,1222784
53,Bretagne,2380594,364724,36077,944052,1035741
75,Nouvelle-Aquitaine,4201774,711017,71629,1641249,1777879


In [81]:
df_grouped['Registered'].sum()

43262592

In [82]:
def plot_referendum_map(referendum_result_by_regions):
    """Plot a map with the results from the referendum.

    * Load the geographic data with geopandas from `regions.geojson`.
    * Merge these info into `referendum_result_by_regions`.
    * Use the method `GeoDataFrame.plot` to display the result map. The results
      should display the rate of 'Choice A' over all expressed ballots.
    * Return a gpd.GeoDataFrame with a column 'ratio' containing the results.
    """
    # Load geographic data
    regions_geo = gpd.read_file("regions.geojson")

    # Calculate the ratio of 'Choice A' over all expressed ballots
    referendum_result_by_regions['ratio'] = referendum_result_by_regions['Choice A'] / (
        referendum_result_by_regions['Choice A'] + referendum_result_by_regions['Choice B']
    )

    # Merge geographic data with referendum results
    merged_data = regions_geo.merge(referendum_result_by_regions, 
                                    left_on="code", right_on="code_reg")

    # Plot the map with the ratio of 'Choice A'
    merged_data.plot(column='ratio', cmap='coolwarm', legend=True, 
                     legend_kwds={'label': "Ratio of Choice A"})
    plt.title("Referendum Results by Region")
    plt.show()

    return merged_data

plot_referendum_map(df_grouped)

ImportError: The 'read_file' function requires the 'pyogrio' or 'fiona' package, but neither is installed or imports correctly.
Importing fiona resulted in: DLL load failed while importing _env: Le module spécifié est introuvable.
Importing pyogrio resulted in: No module named 'pyogrio'